In [1]:
# Variáveis de Controle:
from datetime import datetime
print('Início em: ', datetime.now())

refaz_tabela_doutorados = True

Início em:  2023-04-11 19:47:27.638596


In [2]:
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import unidecode
from bs4 import BeautifulSoup
import lxml
from Lattes import Lattes
from Database import Database

In [3]:
engine = Carga.db_engine()
dt = pd.DataFrame()

In [4]:
db = Database('CNPq')
engine = Carga.db_engine()


In [5]:
#Carregar arquivo na memória
print('Reconstruindo tabela com Ids de doutorandos entrando em 2014. Início em: ', datetime.now())
if refaz_tabela_doutorados:
    print('Carregando indicadores na memória. Início em: ', datetime.now())

    sql = '''
    drop table if exists ids_doutorandos_inicio_em_2014;

    CREATE TABLE ids_doutorandos_inicio_em_2014 AS
        SELECT distinct(id) FROM indicadores AS ind_a where 
         tipo = '@ANO-DE-INICIO DOUTORADO ' AND
         ano = 2014	 and
         (
            SELECT min(ano) from indicadores AS ind_b
             where tipo = '@ANO-DE-INICIO DOUTORADO '
             and ind_b.id = ind_a.id
         ) = 2014;
    '''
    
    db.execute(sql)
    print('Fim em: ', datetime.now())

Reconstruindo tabela com Ids de doutorandos entrando em 2014. Início em:  2023-04-11 19:47:28.974745
Carregando indicadores na memória. Início em:  2023-04-11 19:47:28.975747
Fim em:  2023-04-11 19:55:22.797253


In [6]:
codigo_capes = '045000000000'

In [7]:
#Carregar arquivo na memória
print('Carregando indicadores na memória. Início em: ', datetime.now())

dt = pd.read_sql('''
select 
    ids_doutorandos_inicio_em_2014.id, 
    sum("Valor Pago") AS pagtos 
from ids_doutorandos_inicio_em_2014
    left join dados_gerais
        on ids_doutorandos_inicio_em_2014.id = dados_gerais.id
    left join 
	(select * from pagamentos
		where 1=1
			and "Ano Referência" >= 2014
			and "Ano Referência" < 2019
			and (
				"Modalidade" = 'GD - Doutorado'
				or "Modalidade" = 'GDE - Doutorado no Exterior'
				or "Modalidade" = 'SWE - Doutorado Sanduíche no Exterior'
				or "Modalidade" = 'SWI - Doutorado-Sanduiche Empresarial'
				or "Modalidade" = 'SWP - Doutorado-Sanduiche no Pais')
	 ) AS pagamentos
    on pagamentos."Beneficiário" = dados_gerais.nome

Group by ids_doutorandos_inicio_em_2014.id
''', engine)
dt['capes'] = None
print(f'dt.size: {dt.size}')

print('Término em: ', datetime.now())

Carregando indicadores na memória. Início em:  2023-04-11 19:55:22.827255
dt.size: 75561
Término em:  2023-04-11 19:55:48.624855


In [8]:
dt

,id,pagtos,capes
0,230621439945,NaN,None
1,1074831861726,116730.0,None
2,1171652057144,NaN,None
3,1710013962811,NaN,None
4,2031319204337,NaN,None
...,...,...,...
25182,9997699695074221,NaN,None
25183,9998457346711166,NaN,None
25184,9999386554974089,NaN,None
25185,9999395669687067,NaN,None


In [13]:
financiamentos_capes = []
ids_feitos = 0
total_ids = len(dt)
for index,id_number in dt.iterrows():
    ids_feitos +=1
    print (f'{ids_feitos} / {total_ids}')
    id_number = str(id_number['id'])
    while len(id_number) < 16:
        id_number = '0' + id_number
    lattes = Lattes(id = id_number)
    lattes.read_zip_from_disk()
    if lattes.get_xml():
        lattes.get_dados_gerais_by_xml()
        lattes.insere_dados_gerais_no_bd()
        soup_lattes = BeautifulSoup(lattes.xml, "lxml-xml")
        encontrado = False
        for doutorado in soup_lattes.find_all('DOUTORADO'):
            if 'CODIGO-AGENCIA-FINANCIADORA="045000000000"' in str(doutorado) \
                and 'ANO-DE-INICIO="2014"' in str(doutorado):
                print('Financiado', id_number)
                dt.loc[index,'capes'] = True
                financiamentos_capes.append({'id': id_number, 'CAPES':True, 'encontrado':True})
                
                encontrado = True
                break
        if not encontrado:
            print('Não Financiado', id_number)
            financiamentos_capes.append({'id': id_number, 'CAPES':False, 'encontrado':True})
            dt.loc[index,'capes'] = False
    else:
        print('Lattes não encontrado', id_number)
        financiamentos_capes.append({'id': id_number, 'CAPES':False, 'encontrado':False})   
        dt.loc[index,'capes'] = None


Não Financiado 5158305441051200
13057 / 25187
Financiado 5158495196563414
13058 / 25187
Financiado 5158683640564654
13059 / 25187
Financiado 5159037688140209
13060 / 25187
Não Financiado 5159303385886222
13061 / 25187
Não Financiado 5159465916944082
13062 / 25187
Não Financiado 5160125665750155
13063 / 25187
Não Financiado 5160956195467981
13064 / 25187
Não Financiado 5161025285651971
13065 / 25187
Não Financiado 5161034466253507
13066 / 25187
Financiado 5161628431454077
13067 / 25187
Não Financiado 5162057767120001
13068 / 25187
Não Financiado 5162474598418037
13069 / 25187
Não Financiado 5162494361551709
13070 / 25187
Não Financiado 5162594625699533
13071 / 25187
Não Financiado 5162784228749265
13072 / 25187
Não Financiado 5162902806709375
13073 / 25187
Financiado 5163164382025877
13074 / 25187
Financiado 5163179251798882
13075 / 25187
Financiado 5163690294805547
13076 / 25187
Financiado 5163727526055668
13077 / 25187
Financiado 5163820533827324
13078 / 25187
Não Financiado 516409872

Financiado 5218066625947214
13243 / 25187
Não Financiado 5218841919930284
13244 / 25187
Financiado 5218954387107307
13245 / 25187
Não Financiado 5219949229830368
13246 / 25187
Financiado 5220304160127322
13247 / 25187
Financiado 5221058665417202
13248 / 25187
Financiado 5221375146887164
13249 / 25187
Financiado 5222173687771589
13250 / 25187
Não Financiado 5222323579705936
13251 / 25187
Financiado 5223031749918077
13252 / 25187
Financiado 5223338458784420
13253 / 25187
Financiado 5223550691327862
13254 / 25187
Financiado 5224196102458122
13255 / 25187
Não Financiado 5224203180989285
13256 / 25187
Não Financiado 5224306420021056
13257 / 25187
Financiado 5224441804426291
13258 / 25187
Não Financiado 5224553521824035
13259 / 25187
Financiado 5225066180535475
13260 / 25187
Não Financiado 5225164008360831
13261 / 25187
Não Financiado 5225217690531765
13262 / 25187
Financiado 5226122540955129
13263 / 25187
Financiado 5226605928057153
13264 / 25187
Financiado 5226710191049573
13265 / 25187
Fi

Não Financiado 5277860327434408
13429 / 25187
Financiado 5278086176171090
13430 / 25187
Não Financiado 5279131022736417
13431 / 25187
Não Financiado 5279241638619629
13432 / 25187
Não Financiado 5279352099032473
13433 / 25187
Financiado 5280259610383365
13434 / 25187
Não Financiado 5280442400037283
13435 / 25187
Não Financiado 5280492353472235
13436 / 25187
Financiado 5280869908858897
13437 / 25187
Financiado 5281032215585629
13438 / 25187
Não Financiado 5281301675191832
13439 / 25187
Financiado 5282351930988805
13440 / 25187
Não Financiado 5282418933156256
13441 / 25187
Não Financiado 5282648090059507
13442 / 25187
Financiado 5282667884468325
13443 / 25187
Não Financiado 5283016792353367
13444 / 25187
Não Financiado 5283089824505898
13445 / 25187
Não Financiado 5283183321926399
13446 / 25187
Financiado 5283211843743368
13447 / 25187
Não Financiado 5283454970821787
13448 / 25187
Não Financiado 5284108061391107
13449 / 25187
Não Financiado 5285150768849565
13450 / 25187
Não Financiado 5

Não Financiado 5342051519912757
13614 / 25187
Financiado 5342739718744545
13615 / 25187
Não Financiado 5343090436181227
13616 / 25187
Financiado 5343820532259748
13617 / 25187
Não Financiado 5343996728988850
13618 / 25187
Não Financiado 5344242154652315
13619 / 25187
Não Financiado 5344692835119505
13620 / 25187
Financiado 5345007511863730
13621 / 25187
Não Financiado 5346056851247570
13622 / 25187
Não Financiado 5346220628829577
13623 / 25187
Não Financiado 5346283544017602
13624 / 25187
Não Financiado 5347714470140236
13625 / 25187
Financiado 5347735841246647
13626 / 25187
Financiado 5348108813693753
13627 / 25187
Financiado 5349131138448943
13628 / 25187
Não Financiado 5349191830494068
13629 / 25187
Financiado 5349528715198337
13630 / 25187
Não Financiado 5349782836736032
13631 / 25187
Não Financiado 5350305256049016
13632 / 25187
Não Financiado 5350520951842278
13633 / 25187
Não Financiado 5351178100876194
13634 / 25187
Não Financiado 5351316473311007
13635 / 25187
Financiado 53513

Não Financiado 5429217144821644
13801 / 25187
Financiado 5429537417655258
13802 / 25187
Não Financiado 5429548341600054
13803 / 25187
Financiado 5429738945701879
13804 / 25187
Não Financiado 5430384110605471
13805 / 25187
Não Financiado 5430539572349610
13806 / 25187
Não Financiado 5430690013876567
13807 / 25187
Financiado 5431066770429367
13808 / 25187
Não Financiado 5431176474396060
13809 / 25187
Não Financiado 5431231635299816
13810 / 25187
Financiado 5431729198139323
13811 / 25187
Não Financiado 5431878507016746
13812 / 25187
Financiado 5432074245486639
13813 / 25187
Não Financiado 5432465387302084
13814 / 25187
Não Financiado 5432653356505523
13815 / 25187
Financiado 5433076593233599
13816 / 25187
Financiado 5433221272927171
13817 / 25187
Não Financiado 5433376826611157
13818 / 25187
Não Financiado 5433644301553576
13819 / 25187
Não Financiado 5433951320772360
13820 / 25187
Financiado 5434361183028532
13821 / 25187
Financiado 5434644366406236
13822 / 25187
Não Financiado 543470051

...

Não Financiado 9638799280787687
24353 / 25187
Financiado 9638963035288365
24354 / 25187
Não Financiado 9639470174261573
24355 / 25187
Financiado 9639698628193519
24356 / 25187
Lattes não encontrado 9639853233080917
24357 / 25187
Financiado 9640359700659168
24358 / 25187
Não Financiado 9641557690384438
24359 / 25187
Não Financiado 9642168416851760
24360 / 25187
Não Financiado 9642851209900423
24361 / 25187
Financiado 9643317143990870
24362 / 25187
Não Financiado 9643650982414474
24363 / 25187
Não Financiado 9644279046881992
24364 / 25187
Não Financiado 9645169834284731
24365 / 25187
Não Financiado 9645466196205702
24366 / 25187
Financiado 9645625574666461
24367 / 25187
Não Financiado 9646090589161185
24368 / 25187
Não Financiado 9646299144638951
24369 / 25187
Financiado 9646390577554136
24370 / 25187
Financiado 9646603226938073
24371 / 25187
Não Financiado 9647201064641910
24372 / 25187
Não Financiado 9647518004057701
24373 / 25187
Não Financiado 9647911109416853
24374 / 25187
Não Finan

Não Financiado 9728731779058870
24537 / 25187
Não Financiado 9729794640239166
24538 / 25187
Não Financiado 9730349410522021
24539 / 25187
Não Financiado 9730986978152521
24540 / 25187
Não Financiado 9731415105643926
24541 / 25187
Não Financiado 9732702619172639
24542 / 25187
Financiado 9733072429034502
24543 / 25187
Não Financiado 9733465821156116
24544 / 25187
Financiado 9733548506697828
24545 / 25187
Não Financiado 9733646412301669
24546 / 25187
Financiado 9733655826302766
24547 / 25187
Financiado 9734246972249358
24548 / 25187
Financiado 9734353726034464
24549 / 25187
Não Financiado 9734592726212292
24550 / 25187
Financiado 9735001000722376
24551 / 25187
Financiado 9735120528120188
24552 / 25187
Não Financiado 9735299658686113
24553 / 25187
Não Financiado 9735824812735422
24554 / 25187
Não Financiado 9736184699159629
24555 / 25187
Financiado 9736561625486514
24556 / 25187
Não Financiado 9737977542201724
24557 / 25187
Não Financiado 9738549935676731
24558 / 25187
Não Financiado 97386

Financiado 9806843062179111
24722 / 25187
Não Financiado 9807446940711237
24723 / 25187
Não Financiado 9807500374830473
24724 / 25187
Não Financiado 9807657913811148
24725 / 25187
Financiado 9807741117928143
24726 / 25187
Não Financiado 9807812008418798
24727 / 25187
Financiado 9807906656382737
24728 / 25187
Financiado 9808078161906025
24729 / 25187
Financiado 9808261801310683
24730 / 25187
Não Financiado 9808523308474986
24731 / 25187
Não Financiado 9808975039479271
24732 / 25187
Lattes não encontrado 9809309255681955
24733 / 25187
Financiado 9809524082345584
24734 / 25187
Não Financiado 9809858750740068
24735 / 25187
Não Financiado 9809874053943708
24736 / 25187
Não Financiado 9810369995575569
24737 / 25187
Não Financiado 9811061179562196
24738 / 25187
Financiado 9811171215048475
24739 / 25187
Não Financiado 9811279570941278
24740 / 25187
Não Financiado 9811370847348322
24741 / 25187
Financiado 9811743502935227
24742 / 25187
Financiado 9811824241582252
24743 / 25187
Financiado 981195

24907 / 25187
Financiado 9873820806594531
24908 / 25187
Não Financiado 9875717938382304
24909 / 25187
Não Financiado 9875809504879087
24910 / 25187
Não Financiado 9876194821701111
24911 / 25187
Financiado 9876993628752155
24912 / 25187
Financiado 9877119515800474
24913 / 25187
Não Financiado 9878102867355883
24914 / 25187
Não Financiado 9878381881530579
24915 / 25187
Financiado 9879791700597994
24916 / 25187
Financiado 9880788414949504
24917 / 25187
Financiado 9881337922449588
24918 / 25187
Financiado 9881543977135448
24919 / 25187
Não Financiado 9881586384692906
24920 / 25187
Não Financiado 9882295237244030
24921 / 25187
Não Financiado 9882354004347555
24922 / 25187
Financiado 9882501332652107
24923 / 25187
Não Financiado 9885401483242493
24924 / 25187
Financiado 9885903996929132
24925 / 25187
Não Financiado 9886221945901169
24926 / 25187
Financiado 9886557718033769
24927 / 25187
Não Financiado 9886586761468341
24928 / 25187
Não Financiado 9886725857624920
24929 / 25187
Não Financiado

Não Financiado 9960118188301768
25093 / 25187
Financiado 9960517108883443
25094 / 25187
Financiado 9961782385320076
25095 / 25187
Não Financiado 9962428846020169
25096 / 25187
Não Financiado 9963453541925715
25097 / 25187
Não Financiado 9964206584066834
25098 / 25187
Não Financiado 9964472255865635
25099 / 25187
Não Financiado 9964551986149259
25100 / 25187
Não Financiado 9964748831084241
25101 / 25187
Financiado 9964804222281924
25102 / 25187
Não Financiado 9965219223087021
25103 / 25187
Não Financiado 9965826705584437
25104 / 25187
Não Financiado 9965858205605169
25105 / 25187
Não Financiado 9966077964721005
25106 / 25187
Financiado 9966158817422161
25107 / 25187
Financiado 9966295997259863
25108 / 25187
Não Financiado 9966807856139041
25109 / 25187
Não Financiado 9967842377312411
25110 / 25187
Não Financiado 9967851444577561
25111 / 25187
Não Financiado 9968000130595849
25112 / 25187
Não Financiado 9968575129206926
25113 / 25187
Não Financiado 9968978237628756
25114 / 25187
Financia

In [15]:
dt.to_sql('financiamentos_doutorandos_2014', engine, if_exists='replace')